# Grid search for soccer match result prediction

In [1]:
import numpy as np
import pickle as pkl

In [2]:
import torch
import torch.utils.data

In [3]:
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

In [4]:
import torch.optim as optim

## Data import

In [5]:
soccer_data = pkl.load(open('soccer_tensors.p', 'rb'))

In [6]:
X_train_ts, X_test_ts, y_train_ts, y_test_ts, class_weights_ts = soccer_data

### hyperparameters

hp = {
    'batch_size' : 256, #size of the batch for each gradient step
    'dropout_prob' : 0.5, #probability of dropping a neuron of the hidden layer
    'hl_size' : 50, #size of the hidden layer
    'lr' : 0.0001, #learning rate
    'momentum' : 0.9, #porportion of the previous gradient to add to the new one
    'n_epochs' : 10 #number of loops over the training data for SGD
}

## Model design

In [8]:
"""
inputs: - hp (dict,  hyper parameters)
        - n_tests (integer, the number of tests to make with a given network)
outputs: - mean_acc (float, average accuracy over n_tests runs)
         - std_acc (float, standard deviation of accuracy over n_tests runs)
"""
def get_accuracy(hp, n_tests):
    
    batch_size = hp['batch_size']
    dropout_prob = hp['dropout_prob']
    hl_size = hp['hl_size']
    lr = hp['lr']
    momentum = hp['momentum']
    n_epochs = hp['n_epochs']

    #model design
    class Net(nn.Module):
        def __init__(self):
            super(Net, self).__init__()
            self.fc1 = nn.Linear(614, hl_size)
            self.fc2 = nn.Linear(hl_size, 3)
            self.dropoutLayer = nn.Dropout(p = dropout_prob)

        def forward(self, x):
            x = F.relu(self.fc1(x))
            x = self.dropoutLayer(x)
            x = self.fc2(x)
            return x

    net = Net()
    
    #loss and gradient settings
    criterion = nn.CrossEntropyLoss(weight = class_weights_ts)
    optimizer = optim.SGD(net.parameters(), lr=lr, momentum=momentum)
    
    #building datasets
    trainset = torch.utils.data.TensorDataset(X_train_ts, y_train_ts)
    testset = torch.utils.data.TensorDataset(X_test_ts, y_test_ts)

    trainloader = torch.utils.data.DataLoader(trainset, shuffle=True, batch_size= batch_size)
    testloader = torch.utils.data.DataLoader(testset, shuffle=True, batch_size= batch_size)
    
    #training    
    for epoch in range(n_epochs):  # loop over the dataset multiple times

        for i, data in enumerate(trainloader, 0):
            # get the inputs
            inputs, labels = data

            # wrap them in Variable
            inputs, labels = Variable(inputs), Variable(labels)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

    acc = np.zeros(n_tests)
    for i in range(n_tests):
        correct = 0
        total = 0
        for data in testloader:
            matchs, labels = data
            outputs = net(Variable(matchs))
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum()
        
        acc[i] = 1. * correct / total
        
    return np.mean(acc), np.std(acc)

## Grid design

In [16]:
hp_grid = {
    'batch_size' : [1, 64, 256], #size of the batch for each gradient step
    'dropout_prob' : [0.2, 0.35, 0.5], #probability of dropping a neuron of the hidden layer
    'hl_size' : [50, 150, 300], #size of the hidden layer
    'lr' : [0.0001, 0.0005, 0.001], #learning rate
    'momentum' : [0.8, 0.9, 0.99], #porportion of the previous gradient to add to the new one
    'n_epochs' : [1000] #number of loops over the training data for SGD
}

grid_size = 1
for key, value in hp_grid.items():
    grid_size *= len(value)

In [15]:
"""
keys: set of hyperparameters
values: mean_acc, std_acc
"""
res = {}

In [ ]:
for i in range(grid_size):
    hp = {}
    for key, value in hp_grid.items():
        n = len(value)
        hp[key] = value[i%n]
        i /= n
        
    res[hp] = get_accuracy(hp, 100)